In [1]:
import sys;
sys.path.insert(0, '..')

In [2]:
from math import cos, floor, pi, sqrt, asin, sin

from util import generate_state, print_state_table, is_close, inner

from sim_circuit import *

## Exercise 1

What is the inner product of [i, i] and [-i, i]?

**Answer:**

In [3]:
list1 = [0+1j, 0+1j]
list2 = [0-1j, 0+1j]

sum(list1[k]*list2[k].conjugate() for k in range(len(list1)))

0j

## Exercise 2

Using the code introduced in this section, create a random state with $n = 4$ qubits and apply the classical magnitude amplification procedure for good outcomes 3 and 10.

**Answer:**

In [4]:
def oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1

def inversion(original, current):
    proj = inner(original, current)
    for k in range(len(current)):
        current[k] = 2*proj*original[k] - current[k]

def grover_sim(state, predicate, iterations):
    s = state.copy()

    p = sum([abs(s[k])**2 for k in items])
    theta = asin(sqrt(p))
    assert is_close(inner(s, state), 1)

    for it in range(1, iterations + 1):
        oracle(state, predicate)
        inversion(s, state)
        assert is_close(inner(s, state), cos(2 * it * theta))

        p = sum([abs(state[k])**2 for k in items])
        assert is_close(p, sin((2 * it + 1)*theta)**2) 

In [5]:
n = 4
items = [3, 11]
predicate = lambda i: True if i in items else False

state = generate_state(n)
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000   -0.0839 + i0.1619    0.1824      117.39°   ████                      0.0333
1        0001   -0.1372 - i0.0942    0.1665     -145.47°   ███                       0.0277
2        0010    0.2034 - i0.2349    0.3107      -49.89°   ███████                   0.0966
3        0011   -0.3413 + i0.2044    0.3978      149.80°   █████████                 0.1583
4        0100   -0.0074 + i0.1623    0.1625       92.61°   ███                       0.0264
5        0101   -0.1180 + i0.2386    0.2662      116.31°   ██████                    0.0708
6        0110    0.1036 - i0.2892    0.3071      -70.71°   ███████                   0.0943
7        0111   -0.2391 - i0.2690    0.3599     -131.37°   ████████                  0.1295
8        1000    0.0120 - i0.3210    0.3213      -87.14°   ███████   

In [6]:
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

grover_sim(state, predicate, iterations = 2)

In [7]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000    0.0453 - i0.0873    0.0984      -62.42°   ██                        0.0097
1        0001    0.0740 + i0.0508    0.0898       34.47°   ██                        0.0081
2        0010   -0.1097 + i0.1267    0.1676      130.89°   ████                      0.0281
3        0011   -0.7327 + i0.4387    0.854       149.90°   ████████████████████      0.7294
4        0100    0.0040 - i0.0875    0.0876      -87.62°   ██                        0.0077
5        0101    0.0636 - i0.1287    0.1435      -63.30°   ███                       0.0206
6        0110   -0.0559 + i0.1559    0.1656      109.73°   ███                       0.0274
7        0111    0.1290 + i0.1450    0.1941       48.34°   ████                      0.0377
8        1000   -0.0065 + i0.1731    0.1733       92.15°   ████      

## Exercise 3

Create a magnitude amplification circuit for $n = 3$ qubits and single good outcome 5 using the circuit returned by the function from chapter 4 below as the initial state preparation operator (with `theta = 4*pi/7`).

In [8]:
def prepare_binomial(n, theta):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(len(q)):
        qc.ry(theta, q[i])

    return qc

**Answer:**

In [9]:
def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

In [10]:
def inversion_0_circuit(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.x(q[i])

    qc.mcp(pi, [q[i] for i in range(n - 1)], q[n - 1])

    for i in range(n):
        qc.x(q[i])

    return qc

In [11]:
def inversion_circuit(A):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(A.inverse(), q)

    qc.append(inversion_0_circuit(n), q)

    qc.append(A, q)

    return qc

In [12]:
def grover_iterate_circuit(A, O):
    n = sum(O.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(O, q)

    qc.append(inversion_circuit(A), q)

    return qc

In [13]:
def grover_circuit(A, O, iterations):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(A, q)

    for i in range(1, iterations + 1):
        qc.append(grover_iterate_circuit(A, O), q)
        qc.report(f'iteration_{i}')

    return qc

In [14]:
n = 3
items = [5]
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

theta = 4*pi/7
qc = grover_circuit(prepare_binomial(n, theta), phase_oracle_match(n, items), num_iterations)

In [15]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.0983 + i0.0000    0.0983      180.00°   ██                        0.0097
1        001    -0.1232 + i0.0000    0.1232      180.00°   ██                        0.0152
2        010    -0.1232 + i0.0000    0.1232      180.00°   ██                        0.0152
3        011    -0.1545 + i0.0000    0.1545      180.00°   ███                       0.0239
4        100    -0.1232 + i0.0000    0.1232      180.00°   ██                        0.0152
5        101     0.9271 + i0.0000    0.9271        0.00°   ██████████████████████    0.8595
6        110    -0.1545 + i0.0000    0.1545      180.00°   ███                       0.0239
7        111    -0.1938 + i0.0000    0.1938      180.00°   ████                      0.0375

